In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=False

In [2]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
        
def maskedarray_gt(data, value):
    """ Greater than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data > value] = value
    return data


def maskedarray_lt(data, value):
    """ Less than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data < value] = value
    return data


def ts_fromdebris_func(h, a, b, c):
    """ estimate surface temperature from debris thickness (h is debris thickness, a and k are coefficients) 
        Hill Equation"""
    return a * h**c / (b**c + h**c)


def debris_fromts_func(ts, a, b, c):
    """ estimate debris thickness from surface temperature (ts is surface temperature, a and k are coefficients) 
        Hill Equation"""
    return (ts * b**c / (a - ts))**(1/c)
        
    
def debris_fromts_maskedarray(ts_raw, a, b, c):
    """ Apply debris_fromts_func to masked array
        includes a mask of maximum values, since Michaelis-Mentin Equation has natural maximum 
    Parameters
    ----------
    ts_raw : np.ma.array
        masked array of the unmodified surface temperature
    Returns
    -------
    hd : np.ma.array 
        masked array of the debris thickness (m)
    """
    ts = ts_raw.copy()
    max_value = ts_fromdebris_func(50, a, b, c)
    debris_thick_ts = np.ma.array(maskedarray_gt(ts.data, max_value), mask=np.ma.getmask(ts))
    debris_thick_ts = np.ma.array(maskedarray_lt(ts.data, 0), mask=np.ma.getmask(ts))
    hd = debris_fromts_func(ts.data, a, b, c)
    return hd

In [3]:
# ===== DETERMINE ALL GLACIERS WITH AND WITHOUT OBSERVATIONS =====
hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/'

# Glaciers optimized
glac_tsopt_fns = []
rgiid_list_tsopt = []
for i in os.listdir(hdopt_prms_fp):
    if i.endswith('_hdopt_prms.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            rgiid_list_tsopt.append(i.split('_')[0])            
            glac_tsopt_fns.append(i)
glac_tsopt_fns = sorted(glac_tsopt_fns)
rgiid_list_tsopt = sorted(rgiid_list_tsopt)

main_glac_rgi_tsopt = debris_prms.selectglaciersrgitable(rgiid_list_tsopt)

# All debris-covered glaciers
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_rgiid = sorted([x.split('-')[1] for x in dc_shp.RGIId])
main_glac_rgi_all = debris_prms.selectglaciersrgitable(glac_no=dc_rgiid)

# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

# Add debris stats to area
dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area__1'].values))
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))

main_glac_rgi_tsopt['DC_Area_%'] = main_glac_rgi_tsopt.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_all['DC_Area_%'] = main_glac_rgi_all.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_tsopt['DC_Area_v2'] = main_glac_rgi_tsopt['Area'] * main_glac_rgi_tsopt['DC_Area_%'] / 100
main_glac_rgi_all['DC_Area_v2'] = main_glac_rgi_all['Area'] * main_glac_rgi_all['DC_Area_%'] / 100

# Glaciers lacking optimization
rgiids_missing = set(main_glac_rgi_all.rgino_str.values) - set(main_glac_rgi_tsopt.rgino_str.values)
rgiids_missing = sorted(rgiids_missing)
main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)
main_glac_rgi_missing

835 glaciers in region 1 are included in this model run: ['00013', '00033', '00037', '00038', '00040', '00041', '00042', '00044', '00045', '00046', '00140', '00148', '00242', '00312', '00348', '00351', '00399', '00409', '00426', '00434', '00436', '00537', '00544', '00556', '00557', '00558', '00560', '00561', '00565', '00566', '00569', '00570', '00571', '00572', '00574', '00576', '00578', '00579', '00581', '00582', '00584', '00600', '00660', '00675', '00732', '00739', '00746', '00814', '00852', '00870'] and more
This study is focusing on 835 glaciers in region [1]
6834 glaciers in region 1 are included in this model run: ['00006', '00010', '00012', '00013', '00016', '00017', '00020', '00021', '00022', '00023', '00025', '00027', '00029', '00030', '00032', '00033', '00035', '00036', '00037', '00038', '00040', '00041', '00042', '00044', '00045', '00046', '00068', '00080', '00112', '00118', '00130', '00135', '00138', '00140', '00148', '00151', '00152', '00154', '00159', '00170', '00176', '0

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,5,RGI60-01.00006,-146.244000,63.571000,1,2,10.470,1201,3547,1740,22.0,33,10518,0,0,9,20090703,6,01.00006,1.00006
1,9,RGI60-01.00010,-147.170000,63.513000,1,2,3.806,1280,1998,1666,17.0,35,3332,0,0,9,20090703,10,01.00010,1.00010
2,11,RGI60-01.00012,-146.651000,63.366000,1,2,2.802,1318,2033,1725,17.0,283,3041,0,0,9,20090703,12,01.00012,1.00012
3,15,RGI60-01.00016,-147.557892,63.675205,1,2,13.795,1336,2593,1890,20.0,331,6292,0,0,9,20090703,16,01.00016,1.00016
4,16,RGI60-01.00017,-146.746000,63.459000,1,2,3.765,1416,2351,1924,14.0,238,4019,0,0,9,20090703,17,01.00017,1.00017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,27082,RGI60-01.27087,-143.798000,61.074000,1,5,0.202,1538,1898,1755,21.0,198,872,0,0,9,20100919,27087,01.27087,1.27087
5995,27091,RGI60-01.27096,-140.338000,60.798000,1,5,1.091,2211,2789,2413,18.0,219,2313,0,0,9,20050813,27096,01.27096,1.27096
5996,27095,RGI60-01.27100,-135.610000,59.439000,1,6,1.329,962,1808,1410,24.0,261,2038,0,0,9,20040815,27100,01.27100,1.27100


In [4]:
print('\n', debris_prms.roi + ': calibration includes', main_glac_rgi_tsopt.shape[0], 'glaciers covering',
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum(),1)), 'km2 (' + 
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum() / main_glac_rgi_all['DC_Area_v2'].sum() * 100,1)) + 
      '%) of the total debris-covered glacier area\n')     


 01: calibration includes 835 glaciers covering 5391.9 km2 (77.5%) of the total debris-covered glacier area



In [5]:
# np.where(main_glac_rgi_missing.rgino_str == '18.02342')

In [ ]:
extrap_uncalibrated_glaciers = True
overwrite_hd = False

hd_fp = debris_prms.hd_fp + 'extrap/'
if not os.path.exists(hd_fp):
    os.makedirs(hd_fp)
    
mf_fp = hd_fp + 'meltfactor/'
if not os.path.exists(mf_fp):
    os.makedirs(mf_fp)
    
fig_extrap = debris_prms.output_fig_fp + debris_prms.roi + '/' + 'extrap/'
if not os.path.exists(fig_extrap):
    os.makedirs(fig_extrap)

if extrap_uncalibrated_glaciers:
    
    # ===== NEAREST GLACIERS WITH DATA =====
    n_glac_nearest = 1000
    if n_glac_nearest > main_glac_rgi_tsopt.shape[0]:
        n_glac_nearest = main_glac_rgi_tsopt.shape[0]

    nearest_dict = {}
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[755]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        if glac_idx%500 == 0:
            print(glac_idx, glac_str)
        latlon_dist = (((main_glac_rgi_tsopt['CenLat'].values - main_glac_rgi_missing['CenLat'].values[glac_idx])**2 + 
                        (main_glac_rgi_tsopt['CenLon'].values - main_glac_rgi_missing['CenLon'].values[glac_idx])**2)**0.5)

        latlon_nearidx_list = np.argsort(latlon_dist)[0:n_glac_nearest]
        rgiid_nearest_list = list(main_glac_rgi_tsopt.loc[latlon_nearidx_list,'rgino_str'].values)

        nearest_dict[glac_str] = rgiid_nearest_list

    # Ts filenames
    ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[0]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        rgiid = main_glac_rgi_missing.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        # Ts filename
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        
        # Hd filename
        hd_fn = debris_prms.hd_fn_sample.replace('XXXX', glac_str_noleadzero).replace('.tif','_extrap.tif')

        # Ice thickness filenames
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if ((not os.path.exists(hd_fp + hd_fn) or overwrite_hd) and os.path.exists(thick_dir + thick_fn) and 
            ts_fn not in ['0.0']):
            
            print(glac_idx, glac_str)
            
            # Create glacier feature
            gf = create_glacfeat(thick_dir, thick_fn)
        
            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if not os.path.exists(dc_shp_proj_fn):
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + 
                                            debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + 
                                                    str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_ts=True, ts_fn=ts_fn, gf_add_slope_aspect=False)

            # ===== PLOTS =====
            if debug:
                # DEM
                var_full2plot = gf.z1.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                # Surface temperature
                var_full2plot = gf.ts.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
                # Surface temperature (debris-covered)
                var_full2plot = gf.ts.copy()
                var_full2plot.mask = gf.dc_mask
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
            
            # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
            # Load parameters from nearest neighbor
            rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

            n_nearest = 0
            n_success = 0
            min_n_nearest = 10
            hd_ts_list = []
            mf_list = []
            while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                rgi_str_nearest = rgiid_nearest_list[n_nearest]
                if rgi_str_nearest.startswith('0'):
                    rgi_str_nearest = rgi_str_nearest[1:]
                if debug:
                    print('rgi nearest:', rgi_str_nearest)

                # Load parameters
                df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                # Estimate debris thickness
                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                hd_ma = np.ma.array(hd_array, mask=gf.dc_mask)
                hd_ma_median = np.median(hd_ma.compressed())
                
                if debug:
                    print('hd median:', np.round(hd_ma_median,2))

                # Only include estimates if they are plausible
                if hd_ma_median > 0 and hd_ma_median < debris_prms.hd_max:
                    hd_ts_list.append(hd_ma)
                    n_success += 1

                    # Melt factor
                    mf_array = melt_fromdebris_func(hd_array, func_coeff[0], func_coeff[1]) / melt_cleanice
                    mf_ma = np.ma.array(maskedarray_gt(mf_array, melt_2cm / melt_cleanice), mask=np.ma.getmask(gf.ts))
                    # Linearly interpolate between 0 cm and 2 cm for the melt rate
                    def meltfactor_0to2cm_adjustment(mf, melt_clean, melt_2cm, hd):
                        """ Linearly interpolate melt factors between 0 and 2 cm 
                            based on clean ice and 2 cm sub-debris melt """
                        mf = np.nan_to_num(mf,0)
                        mf[(hd >= 0) & (hd < 0.02)] = (
                            1 + hd[(hd >= 0) & (hd < 0.02)] / 0.02 * (melt_2cm - melt_clean) / melt_clean)
                        return mf
                    mf_ma = np.ma.array(meltfactor_0to2cm_adjustment(mf_ma.data.copy(), melt_cleanice, melt_2cm, 
                                                                     hd_ma.data), mask=np.ma.getmask(gf.ts))
                    mf_list.append(mf_ma)
                        
                n_nearest += 1

            # ===== ESTIMATE DEBRIS THICKNESS FROM ALL COMBINATIONS =====
            if len(hd_ts_list) > 0:
                # DEBRIS THICKNESS based on median of the plausible nearest values
                hd_ts_all = np.ma.array(hd_ts_list)
                hd_ts_med = np.median(hd_ts_all, axis=0)
                gf.debris_thick_ts = hd_ts_med

                if debug:
                    close_fig=False
                else:
                    close_fig=True
                    
                # Debris thickness
                var_full2plot = gf.debris_thick_ts.copy()
                clim = (0,1)
                plot_array(var_full2plot, clim, [gf.glacnum + ' hd (from ts)'], 'inferno', 'hd (m)', 
                           fn=fig_extrap + gf.feat_fn +'_hd_ts.png', close_fig=close_fig)
                
                # MELT FACTOR based on median of the plausible nearest values
                mf_all = np.ma.array(mf_list)
                mf_med = np.median(mf_all, axis=0)
                gf.meltfactor_ts = mf_med
                
                # Melt factor
                var_full2plot = gf.meltfactor_ts.copy()
                clim = (0,1.25)
                plot_array(var_full2plot, clim, [gf.glacnum + ' melt factor'], 'inferno', 'melt factor (-)', 
                           fn=fig_extrap + gf.feat_fn +'_mf.png', close_fig=close_fig)
                
                
                # ===== EXPORT DEBRIS THICKNESS AND MELT FACTOR TIFS ===== 
                # Debris thickness
                gf.debris_thick_ts.mask = gf.dc_mask
                iolib.writeGTiff(gf.debris_thick_ts, hd_fp + hd_fn, gf.ds_dict['z1'])
                
                # Melt factor
                gf.meltfactor_ts.mask = gf.dc_mask
                mf_fn = debris_prms.mf_fn_sample.replace('XXXX',gf.glacnum).replace('.tif','_extrap.tif')
                iolib.writeGTiff(gf.meltfactor_ts, mf_fp + mf_fn, gf.ds_dict['z1'])

                
                # ===== EXPORT THE BINNED DEBRIS THICKNESS AND MELT FACTOR =====
                # Output debris thickness
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                hd_extrap_bin_fp = debris_prms.mb_binned_fp_wdebris_hdts + '../_wdebris_hdts_extrap/'
                if not os.path.exists(hd_extrap_bin_fp):
                    os.makedirs(hd_extrap_bin_fp)
                outbins_df.to_csv(hd_extrap_bin_fp + glac_str + '_mb_bins_hdts_extrap.csv', index=False)
                
            else:
                troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed/' + debris_prms.roi + '/')
                if not os.path.exists(troubleshoot_fp):
                    os.makedirs(troubleshoot_fp)
                txt_fn_extrapfail = glac_str + "-extrap_failed.txt"
                with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                    text_file.write(glac_str + ' failed to any reasonable extrapolation estimates')



0 01.00006
500 01.02378
1000 01.03766
1500 01.05569
2000 01.07585
2500 01.10531
3000 01.12836
3500 01.14564
4000 01.16643
4500 01.18845
5000 01.20928
5500 01.23487
0 01.00006


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/fromnumeric.py:745: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


1 01.00010
2 01.00012
3 01.00016
4 01.00017
5 01.00020
6 01.00021
7 01.00022
8 01.00023
9 01.00025
10 01.00027
11 01.00029
12 01.00030
13 01.00032
14 01.00035
15 01.00036
16 01.00068
17 01.00080
18 01.00112
19 01.00118
20 01.00130
21 01.00135
22 01.00138
23 01.00151
24 01.00152
25 01.00154
26 01.00159
27 01.00170
28 01.00176
29 01.00187
30 01.00189
31 01.00191
32 01.00194
33 01.00196
34 01.00202
35 01.00224
36 01.00225
37 01.00232
38 01.00241
39 01.00243
40 01.00249
41 01.00266
42 01.00269
43 01.00270
44 01.00275
45 01.00291
46 01.00294
47 01.00313
48 01.00319
49 01.00321
50 01.00326
51 01.00336
52 01.00342
53 01.00367
54 01.00368
55 01.00390
56 01.00419
57 01.00425
58 01.00435
59 01.00444
60 01.00475
61 01.00491
62 01.00498
63 01.00501
64 01.00507
65 01.00508
66 01.00533
67 01.00536
68 01.00547
69 01.00552
70 01.00563
71 01.00564
72 01.00583
73 01.00592
74 01.00594
75 01.00595
76 01.00596
77 01.00602
78 01.00611
79 01.00613
80 01.00618
81 01.00621
82 01.00624
83 01.00625
84 01.00627
8

640 01.02841
641 01.02843
642 01.02844
643 01.02846
644 01.02848
645 01.02850
646 01.02856
647 01.02857
648 01.02859
649 01.02861
650 01.02862
651 01.02863
652 01.02865
653 01.02867
654 01.02871
655 01.02874
656 01.02875
657 01.02877
658 01.02888
659 01.02892
660 01.02897
661 01.02901
662 01.02903
663 01.02907
664 01.02908
665 01.02909
666 01.02911
667 01.02913
668 01.02916
669 01.02920
670 01.02923
671 01.02927
672 01.02928
673 01.02929
674 01.02930
675 01.02932
676 01.02935
677 01.02939
678 01.02942
679 01.02944
680 01.02945
681 01.02946
682 01.02951
683 01.02953
684 01.02955
685 01.02957
686 01.02960
687 01.02963
688 01.02964
689 01.02965
690 01.02968
691 01.02972
692 01.02973
693 01.02975
694 01.02978
695 01.02983
696 01.02985
697 01.02991
698 01.02992
699 01.02993
700 01.02995
701 01.02996
702 01.03003
703 01.03006
704 01.03009
705 01.03010
706 01.03011
707 01.03012
708 01.03013
709 01.03018
710 01.03019
711 01.03022
712 01.03023
713 01.03024
714 01.03026
715 01.03027
716 01.03028

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/ma/core.py:4008: RuntimeWarning: invalid value encountered in equal
  check = compare(sdata, odata)


...

1507 01.05579
1508 01.05580
1509 01.05581
1510 01.05582
1511 01.05588
1512 01.05592
1513 01.05608
1514 01.05625
1515 01.05638
1516 01.05639
1517 01.05651
1518 01.05656
1519 01.05672
1520 01.05676
1521 01.05695
1522 01.05708
1523 01.05710
1524 01.05713
1525 01.05720
1526 01.05727
1527 01.05728
1528 01.05745
1529 01.05748
1530 01.05750
1531 01.05756
1532 01.05771
1533 01.05779
1534 01.05781
1535 01.05794
1536 01.05798
1537 01.05803
1538 01.05804
1539 01.05830
1540 01.05836
1541 01.05837
1542 01.05838
1543 01.05845
1544 01.05859
1545 01.05861
1546 01.05862
1547 01.05863
1548 01.05870
1549 01.05871
1550 01.05872
1551 01.05874
1552 01.05875
1553 01.05876
1554 01.05877
1555 01.05880
1556 01.05885
1557 01.05900
1558 01.05903
1559 01.05909
1560 01.05910
1561 01.05912
1562 01.05916
1563 01.05917
1564 01.05918
1565 01.05922
1566 01.05925
1567 01.05936
1568 01.05937
1569 01.05940
1570 01.05943
1571 01.05949
1572 01.05954
1573 01.05958
1574 01.05959
1575 01.05965
1576 01.05968
1577 01.05972
1578 0

2093 01.07906
2094 01.07913
2095 01.07914
2096 01.07915
2097 01.07920
2098 01.07921
2099 01.07926
2100 01.07929
2101 01.07932
2102 01.07934
2103 01.07935
2104 01.07938
2105 01.07939
2106 01.07941
2107 01.07942
2108 01.07945
2109 01.07947
2110 01.07949
2111 01.07957
2112 01.07958
2113 01.07970
2114 01.07975
2115 01.07990
2116 01.07992
2117 01.08007
2118 01.08008
2119 01.08021
2120 01.08027
2121 01.08033
2122 01.08037
2123 01.08043
2124 01.08045
2125 01.08046
2126 01.08052
2127 01.08053
2128 01.08054
2129 01.08055
2130 01.08056
2131 01.08057
2132 01.08058
2133 01.08059
2134 01.08060
2135 01.08062
2136 01.08065
2137 01.08069
2138 01.08071
2139 01.08072
2140 01.08073
2141 01.08074
2142 01.08075
2143 01.08076
2144 01.08077
2145 01.08080
2146 01.08081
2147 01.08082
2148 01.08083
2149 01.08084
2150 01.08085
2151 01.08087
2152 01.08088
2153 01.08089
2154 01.08090
2155 01.08093
2156 01.08094
2157 01.08099
2158 01.08100
2159 01.08101
2160 01.08102
2161 01.08104
2162 01.08108
2163 01.08109
2164 0

2681 01.11491
2682 01.11495
2683 01.11496
2684 01.11498
2685 01.11500
2686 01.11501
2687 01.11502
2688 01.11505
2689 01.11506
2690 01.11507
2691 01.11508
2692 01.11510
2693 01.11511
2694 01.11512
2695 01.11513
2696 01.11516
2697 01.11517
2698 01.11518
2699 01.11519
2700 01.11520
2701 01.11521
2702 01.11525
2703 01.11526
2704 01.11527
2705 01.11529
2706 01.11530
2707 01.11532
2708 01.11533
2709 01.11534
2710 01.11535
2711 01.11539
2712 01.11540
2713 01.11543
2714 01.11547
2715 01.11549
2716 01.11551
2717 01.11552
2718 01.11554
2719 01.11555
2720 01.11556
2721 01.11557
2722 01.11561
2723 01.11562
2724 01.11566
2725 01.11577
2726 01.11578
2727 01.11579
2728 01.11580
2729 01.11581
2730 01.11582
2731 01.11583
2732 01.11584
2733 01.11585
2734 01.11586
2735 01.11589
2736 01.11595
2737 01.11598
2738 01.11601
2739 01.11602
2740 01.11604
2741 01.11607
2742 01.11610
2743 01.11611
2744 01.11617
2745 01.11621
2746 01.11622
2747 01.11624
2748 01.11629
2749 01.11630
2750 01.11631
2751 01.11634
2752 0

3267 01.13722
3268 01.13723
3269 01.13724
3270 01.13726
3271 01.13728
3272 01.13729
3273 01.13732
3274 01.13734
3275 01.13735
3276 01.13736
3277 01.13737
3278 01.13739
3279 01.13740
3280 01.13741
3281 01.13742
3282 01.13743
3283 01.13744
3284 01.13765
3285 01.13766
3286 01.13768
3287 01.13776
3288 01.13779
3289 01.13781
3290 01.13782
3291 01.13783
3292 01.13787
3293 01.13794
3294 01.13796
3295 01.13803
3296 01.13807
3297 01.13809
3298 01.13812
3299 01.13814
3300 01.13815
3301 01.13816
3302 01.13817
3303 01.13833
3304 01.13838
3305 01.13841
3306 01.13848
3307 01.13855
3308 01.13856
3309 01.13860
3310 01.13864
3311 01.13866
3312 01.13873
3313 01.13882
3314 01.13883
3315 01.13885
3316 01.13893
3317 01.13900
3318 01.13912
3319 01.13916
3320 01.13918
3321 01.13920
3322 01.13923
3323 01.13930
3324 01.13945
3325 01.13946
3326 01.13947
3327 01.13948
3328 01.13949
3329 01.13950
3330 01.13958
3331 01.13964
3332 01.13965
3333 01.13968
3334 01.13980
3335 01.13982
3336 01.13984
3337 01.13986
3338 0

3853 01.16251
3854 01.16257
3855 01.16263
3856 01.16264
3857 01.16265
3858 01.16274
3859 01.16275
3860 01.16276
3861 01.16279
3862 01.16281
3863 01.16288
3864 01.16292
3865 01.16293
3866 01.16294
3867 01.16300
3868 01.16309
3869 01.16310
3870 01.16311
3871 01.16312
3872 01.16313
3873 01.16314
3874 01.16318
3875 01.16328
3876 01.16334
3877 01.16337
3878 01.16343
3879 01.16344
3880 01.16348
3881 01.16352
3882 01.16365
3883 01.16370
3884 01.16380
3885 01.16390
3886 01.16391
3887 01.16392
3888 01.16393
3889 01.16394
3890 01.16395
3891 01.16396
3892 01.16397
3893 01.16398
3894 01.16400
3895 01.16401
3896 01.16402
3897 01.16403
3898 01.16404
3899 01.16407
3900 01.16408
3901 01.16409
3902 01.16410
3903 01.16415
3904 01.16416
3905 01.16417
3906 01.16418
3907 01.16420
3908 01.16429
3909 01.16430
3910 01.16431
3911 01.16432
3912 01.16435
3913 01.16436
3914 01.16437
3915 01.16441
3916 01.16448
3917 01.16449
3918 01.16450
3919 01.16452
3920 01.16453
3921 01.16457
3922 01.16458
3923 01.16459
3924 0

In [ ]:
print('\nDONE!\n')

In [ ]:
print('TO-DO LIST:')
print('  - Mosaic extrapolated and calibrated glaciers into 1-degree pixels')
print('  - Test robustness on glaciers with data (Ngozumpa, Miage, etc.)')